In [230]:
import numpy as np
from numpy.linalg import norm, inv
from scipy.special import comb
from scipy.spatial.distance import cosine

In [231]:
def score(model, predicted_model):
    model_mask = model != 0
    predicted_model_mask = predicted_model != 0
    dim_score = np.sum(model_mask) == np.sum(predicted_model_mask)
    support_score = np.sum(model_mask * predicted_model_mask) >= np.sum(model_mask) - 3
    similarity = cosine(X @ model, X @ predicted_model)
    if np.isnan(similarity):
        similarity = 0
    estimation_score = similarity > 0.9
    return {'Dimension': dim_score, 'Support': support_score, 'Estimation': estimation_score}

def masked_lse(X, Y, mask):
    X_ = X[:, mask]
    beta_ = inv(X_.T @ X_) @ X_.T @ Y
    beta = np.zeros(X.shape[1])
    beta[mask] = beta_
    return beta

In [232]:
n = p = 100
X = np.eye(n, p)
beta = np.zeros(p)
beta[:10] = 1
num_simulations = 1000

In [233]:
success = {'Cp': [], 'BIC': []}
for i in range(num_simulations):
    Y = X @ beta + np.random.standard_normal(p)
    cp_model = masked_lse(X, Y, np.square(X @ Y) > 2)
    bic_model = masked_lse(X, Y, np.square(X @ Y) > np.log(n))
    success['Cp'].append(score(beta, cp_model))
    success['BIC'].append(score(beta, bic_model))
success_rate = {}
for criterion in ('Cp', 'BIC'):
    success_rate[criterion] = {}
    for s in ('Dimension', 'Support', 'Estimation'):
        success_rate[criterion][s] = np.mean([cs[s] for cs in success[criterion]])
print('Cp\t', success_rate['Cp'])
print('BIC\t', success_rate['BIC'])

Cp	 {'Dimension': 0.024, 'Support': 0.026, 'Estimation': 0.095}
BIC	 {'Dimension': 0.003, 'Support': 0.0, 'Estimation': 0.259}


In [234]:
def metropolis_hastings(X, Y, beta_0, b, K=0.01, max_iter=200):
    p = beta_0.shape[0]
    beta = np.empty(shape=(max_iter + 1, p))
    beta[0, :] = np.copy(beta_0)
    dim = np.sum(beta_0 != 0)
    pi = 1 / (p * comb(p, dim))
    r = np.square(norm(X @ beta_0 - Y)) + K * np.square(np.sqrt(dim) + np.sqrt(2 * np.log(1 / pi)))
    for i in range(max_iter):
        j = np.random.randint(p)
        beta_new = np.copy(beta[i])
        beta_new[j] = 1 if beta[i][j] == 0 else 0
        dim = np.sum(beta_new != 0)
        pi_new = 1 / (p * comb(p, dim))
        r_new = np.square(norm(X @ beta_new - Y)) + K * np.square(np.sqrt(dim) + np.sqrt(2 * np.log(1 / pi_new)))
        pr = (pi_new * np.exp(-b * r_new)) / (pi * np.exp(-b * r))
        if np.random.uniform() < pr:
            beta[i + 1] = beta_new
            pi = pi_new
            r = r_new
        else:
            beta[i + 1] = np.copy(beta[i])
    return np.mean(beta[1:], axis=0)

In [235]:
def estimation_error():
    f0_error, cp_error, bic_error, mh1_error, mh3_error, mh7_error = 0, 0, 0, 0, 0, 0
    for i in range(num_simulations):
        Y = X @ beta + np.random.standard_normal(p)
        f0 = np.zeros(X.shape[1])
        cp_model = masked_lse(X, Y, np.square(X @ Y) > 2)
        bic_model = masked_lse(X, Y, np.square(X @ Y) > np.log(n))
        mh1_model = metropolis_hastings(X, Y, np.zeros(p), 0.1)
        mh3_model = metropolis_hastings(X, Y, np.zeros(p), 0.3)
        mh7_model = metropolis_hastings(X, Y, np.zeros(p), 0.7)
        f0_error += np.square(norm(f0 - beta))
        cp_error += np.square(norm(cp_model - beta))
        bic_error += np.square(norm(bic_model - beta))
        mh1_error += np.square(norm(mh1_model - beta))
        mh3_error += np.square(norm(mh3_model - beta))
        mh7_error += np.square(norm(mh7_model - beta))
    f0_error /= num_simulations
    cp_error /= num_simulations
    bic_error /= num_simulations
    mh1_error /= num_simulations
    mh3_error /= num_simulations
    mh7_error /= num_simulations
    print(f'{"":10} {"":10} {"":10} {"Metropolis Hastings Error":30}')
    print(f'{"F0 Error":10}|{"Cp Error":10}|{"BIC Error":10}|{"b=0.1":10}|{"b=0.3":10}|{"b=0.7":10}')
    print(f'{f0_error:10f}|{cp_error:10f}|{bic_error:10f}|{mh1_error:10f}|{mh3_error:10f}|{mh7_error:10f}')

In [236]:
estimation_error()

                                 Metropolis Hastings Error     
F0 Error  |Cp Error  |BIC Error |b=0.1     |b=0.3     |b=0.7     
 10.000000| 63.241762| 30.353245| 10.178400| 10.080962|  9.843140


In [237]:
beta[:10] = 2
estimation_error()

                                 Metropolis Hastings Error     
F0 Error  |Cp Error  |BIC Error |b=0.1     |b=0.3     |b=0.7     
 40.000000| 67.998300| 45.715416| 39.925855| 39.350322| 34.987817
